Directories and files to change (run all below after executing)


In [ ]:

import os
import numpy as np
from scipy.io import loadmat

xpos = loadmat(r'N:\MICROSCOPE\Tom\CODE\LC code\processed mat + csv files\06August\xpos_06August_sess18.mat')['xpos'].flatten() #where the spikeSorting x position .mat files is located
ypos = loadmat(r'N:\MICROSCOPE\Tom\CODE\LC code\processed mat + csv files\06August\ypos_06August_sess18.mat')['ypos'].flatten() #... y position

os.chdir(r"N:\MICROSCOPE\Tom\CODE\LC code\processed python files\Jul30 herbs to ccf\Sess 18 20250305_M335_g0 L hand L record") #the directory where you want to load pkl files save all outputs

file_path = ['probe_1_R_cy5.pkl',                 #the names of the pkl files for the current session 
             'probe_2_R_cy5.pkl',
             'probe_3_R_cy5.pkl',
             'probe_4_R_cy5.pkl'
]


hemisphere_backwards = False #if by accident you put the shanks of a left hemisphere recording in the right hemisphere on herbs, change this to True

import pkl file


In [ ]:
import pickle


# Load the pickle files, always ant - post

             
data_site_locs = []
data_rel_site_locs = []
data_voxels = []
data_ccf_sites = []
for i in file_path:
    with open(i, 'rb') as f:
        loaded_data = pickle.load(f)
        data_site_locs.append(loaded_data['data']['sites_loc_b'])

        # print what location in data_site_locs that just went into 
        print(f"Appended to data_site_locs[{len(data_site_locs)-1}] from file: {i}")
        data = loaded_data['data']


        data_rel_site_locs.append(loaded_data['data']['sites_loc_relative'])

        data_voxels.append(loaded_data['data']['sites_vox'])

        data_ccf_sites.append(loaded_data['data']['sites_label'])


flip ccf x if necessary 

In [ ]:
#iterate through data_site_locs and change the x values to the negative of the x values
if hemisphere_backwards == True:
    print("flip")
    for shank in data_site_locs:
        for col in shank:
            temp1 = shank
            temp2 = col
            col[:, 0] = -col[:, 0]

    for shank in data_voxels:
        for col in shank:
            temp1 = shank
            temp2 = col
            col[:, 0] = 1140 - col[:, 0]

break up to corresponding xyz positions

In [ ]:
import plotly.graph_objs as go
import plotly.offline as pyo

fig_plotly = go.Figure()

# Plot all 3D positions for every probe and both columns in data_site_locs
for probe_idx, probe in enumerate(data_site_locs):
    for col_idx, arr in enumerate(probe):
        fig_plotly.add_trace(go.Scatter3d(
            x=arr[:, 0], y=arr[:, 1], z=arr[:, 2],
            mode='markers',
            marker=dict(size=3),
            name=f'probe {probe_idx+1} col {col_idx+1}',
        ))

fig_plotly.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    ),
    title='3D positions from all entries in data_site_locs'
)

# Save to HTML file
pyo.plot(fig_plotly, filename='probe_positions_3d.html', auto_open=False)


# always label the probes as UP in HERBS. Never left or right. 

adjust data_rel_site_locs values so the lowest y is zero (so subtract [-1] and add 8 to the xvals 

In [ ]:
import numpy as np

for shank in range(4):
    for col in range(2):

        #make bottom (y) zero
        data_rel_site_locs[shank][col][:,0] -= np.min(data_rel_site_locs[shank][col][:,0])

        data_rel_site_locs[shank][col][:,0] = (data_rel_site_locs[shank][col][:,0] - np.max(data_rel_site_locs[shank][col][:,0])) *-1




        #space out shanks
        x_vals = data_rel_site_locs[shank][col][:,1]
        x_vals[x_vals == 16] = 32 + 250*shank
        x_vals[x_vals == -8] = 0 + 250*shank 
        data_rel_site_locs[shank][col][:,1] = x_vals



temp = data_rel_site_locs[0][0]
temp2 = data_ccf_sites[0][0]

load x, y pos 


In [ ]:

import numpy as np
import matplotlib.pyplot as plt

if 1 == 0:
    xpos = xpos.astype(np.int32)  # Cast to signed integer to avoid overflow (older xpos files have this issue)
    
fig = plt.figure(figsize = (5,10))
plt.scatter(xpos, ypos)
plt.show()

print(np.min(xpos), np.max(xpos), np.min(ypos), np.max(ypos))


assign units their ccf location based on their xy posn from spike sorting that matches the above the best

In [ ]:
from math import floor

#create array of len xpos that will hold 3d coordinates for each unit
unit_3d_coords = np.zeros((len(xpos), 3))
unit_voxels = np.zeros((len(xpos),3))
unit_sites = np.zeros((len(xpos)))



for unit in range(len(xpos)):
    cur_xpos = xpos[unit]

    #round cur_xpos_idx to nearest 250
    cur_xpos_idx = round(cur_xpos / 250)

    #
    
    cur_col_idx = floor((cur_xpos - 250*cur_xpos_idx) / 32)
    #print(cur_xpos, cur_xpos_idx, 250*cur_xpos_idx, cur_col_idx)
    if cur_col_idx > 1: 
        cur_col_idx = 1
    if cur_col_idx < 0:
        cur_col_idx = 0

    if(data_voxels[cur_xpos_idx][0][0][1] < data_voxels[cur_xpos_idx][1][0][1]): #flip columns if more anterior
        #print(data_voxels[cur_xpos_idx][0][0][1], data_voxels[cur_xpos_idx][1][0][1])
        cur_col_idx = 1 - cur_col_idx

    # if col_flip[cur_xpos_idx]: #relic from manual flipping of columns
    #    print(cur_xpos_idx)
    #    cur_col_idx = 1 - cur_col_idx  

    cur_shank = data_rel_site_locs[cur_xpos_idx]
    cur_col = cur_shank[cur_col_idx]

    cur_ypos = ypos[unit]

    #find the column in cur_shank that has the closest y value to cur_ypos
    closest_col = None
    closest_diff = float('inf')
    for col_idx, col_val in enumerate(cur_col[:,0]):
        diff = abs(col_val - cur_ypos)
        #print(col_val, cur_ypos, diff)
        if diff < closest_diff:
            closest_diff = diff
            closest_col = col_idx

    #get 3d coords from data_site_locs
    curr_3d_coord = data_site_locs[cur_xpos_idx][cur_col_idx][closest_col]
    unit_3d_coords[unit] = curr_3d_coord

    unit_voxels[unit] = data_voxels[cur_xpos_idx][cur_col_idx][closest_col]
    unit_sites[unit] = int(data_ccf_sites[cur_xpos_idx][cur_col_idx][closest_col])
    #print(f"Unit {unit} xpos {xpos[unit]} ypos {ypos[unit]} 3D coords: {curr_3d_coord}, voxel: {unit_voxels[unit]}, site: {unit_sites[unit]}")


3d plot of unit_3d_coords

In [ ]:
fig_plotly = go.Figure()

# Plot all 3D positions for every probe and both columns in data_site_locs (blue)
for probe_idx, probe in enumerate(data_site_locs):
    for col_idx, arr in enumerate(probe):
        fig_plotly.add_trace(go.Scatter3d(
            x=arr[:, 0], y=arr[:, 1], z=arr[:, 2],
            mode='markers',
            marker=dict(size=3, color='blue'),
            name=f'probe {probe_idx+1} col {col_idx+1}',
        ))

# Plot unit_3d_coords in red
fig_plotly.add_trace(go.Scatter3d(
    x=unit_3d_coords[:, 0], y=unit_3d_coords[:, 1], z=unit_3d_coords[:, 2],
    mode='markers',
    marker=dict(size=4, color='red'),


    name='unit_3d_coords'
))

fig_plotly.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    ),
    title='3D positions: probes (blue) and units (red)'
)

pyo.plot(fig_plotly, filename='3dpositions_probes_and_units.html', auto_open=False)

plot x y of unit_3d_coords, xpos ypos as well, with the same colour per point

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.lines import Line2D

fig, ax = plt.subplots(2, 1, figsize=(10, 10))

# Map unique unit_sites to evenly spaced colors across the full colormap
unique_sites = np.unique(unit_sites)
n_sites = len(unique_sites)
site_to_color_idx = {site: i for i, site in enumerate(unique_sites)}
norm = plt.Normalize(0, n_sites - 1)
cmap = plt.get_cmap('turbo')


# Assign each unit a color index based on its site
color_indices = np.array([site_to_color_idx[site] for site in unit_sites])


# Scatter plot with color mapping by evenly spaced site indices
sc = ax[0].scatter(
    unit_3d_coords[:, 1], unit_3d_coords[:, 2],
    c=color_indices, s=22, cmap='turbo'
)
ax[0].set_xlabel('ccf Y')
ax[0].set_ylabel('ccf Z')


ax[0].set_xlim([np.max(unit_3d_coords[:,1]) + 10, np.min(unit_3d_coords[:,1]) - 20])
ax[0].set_title('3D view of unit_3d_coords')

# For legend, use evenly spaced colors
legend_elements = [
    Line2D([0], [0], marker='o', color='w', label=f'Site {site:.0f}',
           markerfacecolor=cmap(norm(i)), markersize=8)
    for i, site in enumerate(unique_sites)
]

ax[0].legend(handles=legend_elements, title='Unit Sites', loc = 'upper right')

# Use the same color indices and colormap for ax[1] as for ax[0]
sc2 = ax[1].scatter(
    xpos, ypos, c=color_indices, cmap='turbo', s=4, label='xpos ypos'
)
for i in [0, 32, 32+32, 250, 282, 282+32, 500, 532, 532+32, 750, 782, 782+32]:
    ax[1].axvline(i)
ax[1].set_xlabel('.mat xpos')
ax[1].set_ylabel('.mat ypos')
ax[1].set_xlim([-100, 1150])
ax[1].legend(handles=legend_elements, title='Unit Sites', loc='upper right')


plt.show()


#save image
fig.savefig('unit_sites_3d_and_xpos_ypos.png', dpi=300, bbox_inches='tight')

Change voxels 

In [ ]:
#currently, voxels x is ML L-R, Y is P-A, Z is V-D
width_x = 1140
width_y = 1320
width_z = 800
resolution = 0.1 #10 um per voxel

unit_voxels [:,2] = width_z - unit_voxels[:,2]
unit_voxels[:,1] = width_y - unit_voxels[:,1]

#rearrange so that x,y,z in unit_voxels is AP, DV, LR
unit_voxels = unit_voxels[:, [1, 2, 0]] 

unit_voxels = unit_voxels/resolution



saving

In [ ]:
#os.chdir(r"D:\HDD Storage\Jul9 herbs to ccf\Sess 1 20250305_M331_g0 L hand L record")

np.save('unit_sites.npy', unit_sites)
np.save('unit_3d_coords.npy', unit_3d_coords)
np.save('unit_voxels.npy', unit_voxels)


#save as .mat files
from scipy.io import savemat
savemat('unit_sites.mat', {'unit_sites': unit_sites})
savemat('unit_3d_coords.mat', {'unit_3d_coords': unit_3d_coords})
savemat('unit_voxels.mat', {'unit_voxels': unit_voxels})

Check: First shank and last shank AP ML DV Makes sense

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # Necessary for 3D plotting


# Get indices of units on first and last shanks
just_first_shanks = np.where(xpos <= 50)[0]
just_last_shanks = np.where(xpos >= 750)[0]

# Get 3D coordinates of those units
voxels_first = unit_voxels[just_first_shanks]
voxels_last = unit_voxels[just_last_shanks]

# Create 3D plot
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Plot first shank units in blue
ax.scatter(voxels_first[:, 0], voxels_first[:, 1], voxels_first[:, 2], c='blue', label='First Shank')

# Plot last shank units in red
ax.scatter(voxels_last[:, 0], voxels_last[:, 1], voxels_last[:, 2], c='red', label='Last Shank')

# Labels and legend
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.legend()
plt.title("3D Positions of Units on First and Last Shanks")
plt.show()